## Import

In [124]:
import random
import wget
import os
import torch
import pandas as pd
import numpy as np

from enum import Enum
from torch.optim import AdamW
from torch.utils.data import random_split, TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, BertTokenizer, AutoModelForMaskedLM, BertForSequenceClassification, \
                         get_linear_schedule_with_warmup

print('Import is done.')

Import is done.


## Configuration

In [125]:
class Configurator:
    def __init__(self) -> None:
        self._params = {
            'random.seed': 42,
            
            'url.dataset.train': 'https://github.com/RussianNLP/RuCoLA/blob/main/data/in_domain_train.csv?raw=true',
            'url.dataset.test': 'https://github.com/RussianNLP/RuCoLA/blob/main/data/in_domain_dev.csv?raw=true',
            
            'path.dataset.train': './train_dataset.csv',
            'path.dataset.test': './test_dataset.csv',
            
            'name.train': 'TRAIN',
            'name.test': 'TEST',
  
            'dataframe.train.names': ['id', 'sentence', 'acceptable', 'error_type', 'detailed_source'],
            'dataframe.test.names': ['id', 'sentence', 'acceptable', 'error_type', 'detailed_source'],
            
            'bert.train-size': 0.9,
            'bert.batch-size': 32,
            'bert.optimizer.learning-rate': 2e-5,
            'bert.optimizer.eps': 1e-8,
            'bert.train.epochs': 2
        }
    
    def __call__(self, *args, **kwargs):
        if len(args) == 0 or args[0] not in self._params:
            return None
        return self._params[args[0]]
    
    def check(self, *args):
        result = True
        absence_params = set()
        for arg in args:
            if isinstance(arg, str) and arg not in self._params:
                result = False
                absence_params.add(arg)
        message = 'Absence params: ' + ', '.join(absence_params)
        assert result, message
        
        
conf = Configurator()

print('Configuration block is done.')

Configuration block is done.


## Conditions

In [130]:
class CList(Enum):
    SET_RANDOM_SEED = 0
    DEVICE_DEFINED = 1
    DATASET_DOWNLOADED = 2
    DATASET_LOADED = 3
    DATASET_PREPARED = 4
    TOKENIZERS_MODELS_CREATED = 5
    BERT_SCHEDULER_CREATED = 6

class Conditions:    
    def __init__(self) -> None:
        self._conditions = {}
        
    def set(self, message, *conditions):
        for condition in conditions:
            self._conditions[condition] = True
        print(message)
    
    def check(self, *conditions):
        result = True
        absence = set()
        for condition in conditions:
            if condition not in self._conditions:
                result = False
                absence.add(condition.name)
        fail_message = 'Absence conditions: ' + ', '.join(absence)
        assert result, fail_message

conds = Conditions()

print('Conditions block is done.')

Conditions block is done.


## Set random seed

In [131]:
conf.check('random.seed')

random.seed(conf('random.seed'))
np.random.seed(conf('random.seed'))
torch.manual_seed(conf('random.seed'))
torch.cuda.manual_seed(conf('random.seed'))

conds.set('Random seed is set.', CList.SET_RANDOM_SEED)

Random seed is set.


## Define device

In [132]:
conds.check(CList.SET_RANDOM_SEED)

if torch.cuda.is_available():
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print(f'We will use the GPU: {torch.cuda.get_device_name(0)}')
    device = torch.device('cuda')
else:
    print('No GPU available, using the GPU instead.')
    device = torch.device('cpu')
    
conds.set("Device is defined", CList.DEVICE_DEFINED)

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 3080 Laptop GPU
Device is defined


## Models & tokenizers & optimizers creation

In [133]:
conf.check('bert.optimizer.learning-rate', 'bert.optimizer.eps')
conds.check(CList.DEVICE_DEFINED)

bert_tokenizer = BertTokenizer.from_pretrained("ai-forever/ruBert-base")
bert_model = BertForSequenceClassification.from_pretrained(
    "ai-forever/ruBert-base",
    num_labels=2,
    output_attentions=False,
    output_hidden_states=False
)
bert_optimizer = AdamW(bert_model.parameters(), lr=conf('bert.optimizer.learning-rate'), eps=conf('bert.optimizer.eps'))

conds.set('Models & tokenizers creation is done.', CList.TOKENIZERS_MODELS_CREATED)

Some weights of the model checkpoint at ai-forever/ruBert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not ini

Models & tokenizers creation is done.


## Downloading datasets on disk

In [134]:
# optimize checking  !!!
conf.check(
    'url.dataset.train',
    'path.dataset.train',
    'name.train',
    'url.dataset.test',
    'path.dataset.test',
    'name.test'
);
conds.check(CList.TOKENIZERS_MODELS_CREATED)

def load_dataset(url: str, path: str, name: str):
    if os.path.exists(path):
        print('Dataset "' + name + '" is already downloaded.')
    else:
        wget.download(url, path)
        print(' Dataset "' + name + '" is downloaded.')
        
load_dataset(conf('url.dataset.train'), conf('path.dataset.train'), conf('name.train'))
load_dataset(conf('url.dataset.test'), conf('path.dataset.test'), conf('name.test'))
conds.set('Datasets downloading is done', CList.DATASET_DOWNLOADED)

Dataset "TRAIN" is already downloaded.
Dataset "TEST" is already downloaded.
Datasets downloading is done


## Loading datasets from disk

In [107]:
conf.check(
    'path.dataset.train',
    'path.dataset.test',
    'dataframe.train.names',
    'dataframe.test.names'
)
conds.check(Conditions.DATASET_DOWNLOADED)

train_dataframe = pd.read_csv(
    conf('path.dataset.train'),
    names=conf('dataframe.train.names'),
    skiprows=1,
    usecols=conf('dataframe.train.usecols')
)

test_dataframe = pd.read_csv(
    conf('path.dataset.test'),
    names=conf('dataframe.test.names'),
    skiprows=1,
    usecols=conf('dataframe.test.usecols')
)
conds.set(Conditions.DATASET_LOADED)

print('Datasets are loaded from disk.')

Datasets are loaded from disk.


## BERT datasets preparation, create dataloaders

In [108]:
conf.check('bert.batch-size', 'bert.batch-size', 'bert.batch-size')
conds.check(Conditions.DATASET_LOADED)

train_sentences = train_dataframe.sentence.values
train_acceptables = train_dataframe.acceptable.values
test_sentences = test_dataframe.sentence.values
test_acceptables = test_dataframe.acceptable.values


def define_raw_max_length_by_bert(sentences, raw_max_length):
    for sentence in sentences:
        input_ids = bert_tokenizer.encode(sentence, add_special_tokens=True)
        raw_max_length = max(raw_max_length, len(input_ids))    
    return raw_max_length


raw_max_length = define_raw_max_length_by_bert(train_sentences, 0)
raw_max_length = define_raw_max_length_by_bert(test_sentences, raw_max_length)


def define_max_length(raw_max_length, threshold):
    return threshold if threshold >= raw_max_length else define_max_length(raw_max_length, threshold * 2)


max_length = define_max_length(raw_max_length, 1)

def create_bert_dataset(sentences, acceptables, max_length):
    input_ids = []
    attention_masks = []
    for sentence in sentences:
        encoded_dict = bert_tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            max_length=max_length,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt'
        )
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    acceptables = torch.tensor(acceptables)

    return TensorDataset(input_ids, attention_masks, acceptables)

train_val_dataset = create_bert_dataset(train_sentences, train_acceptables, max_length)
test_dataset = create_bert_dataset(test_sentences, test_acceptables, max_length);

train_val_dataset_size = len(train_val_dataset)
train_size = int(conf('bert.train-size') * train_val_dataset_size)
val_size = train_val_dataset_size - train_size
train_dataset, val_dataset = random_split(train_val_dataset, [train_size, val_size])

train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=conf('bert.batch-size'))
val_dataloader = DataLoader(val_dataset, sampler=SequentialSampler(val_dataset), batch_size=conf('bert.batch-size'))
test_dataloader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size=conf('bert.batch-size'))

conds.set(Conditions.DATASET_PREPARED)
print('BERT datasets are prepared.')

BERT datasets are prepared.


## BERT scheduler creation

In [109]:
conf.check('bert.train.epochs')
conds.check(Conditions.DATASET_PREPARED)

bert_scheduler = get_linear_schedule_with_warmup(
    bert_optimizer,
    num_warmup_steps=0,
    num_training_steps=len(train_dataloader) * conf('bert.train.epochs')
)

conds.set(Conditions.BERT_SCHEDULER_CREATED)
print('BERT scheduler is created.')

BERT scheduler is created.


## BERT finetune training

In [ ]:
conds.check(Conditions.BERT_SCHEDULER_CREATED)


#     def flat_accuracy(predictions, labels):
#         predictions_flat = np.argmax(predictions, axis=1).flatten()
#         labels_flat = labels.flatten()
#         return np.sum(predictions_flat == labels_flat) / len(labels_flat)

#     def format_time(elapsed):
#         elapsed_rounded = int(round(elapsed))
#         return str(datetime.timedelta(seconds=elapsed_rounded))

#     print_header('fix seed')
#     seed_val = 42
#     random.seed(seed_val)
#     np.random.seed(seed_val)
#     torch.manual_seed(seed_val)
#     torch.cuda.manual_seed(seed_val)

#     print_header('train/valid')
#     training_stats = []
#     total_t0 = time.time()

#     for epoch_i in range(0, epochs):
#         print("")
#         print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
#         print('Training...')

#         t0 = time.time()
#         total_train_loss = 0

#         model.train()

#         for step, batch in enumerate(train_dataloader):
#             if step % 40 == 0 and not step == 0:
#                 elapsed = format_time(time.time() - t0)
#                 print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

#             b_input_ids = batch[0].to(device)
#             b_input_mask = batch[1].to(device)
#             b_labels = batch[2].to(device)

#             model.zero_grad()

#             res = model(
#                 b_input_ids,
#                 token_type_ids=None,
#                 attention_mask=b_input_mask,
#                 labels=b_labels
#             )
#             loss = res['loss']
#             logits = res['logits']

#             total_train_loss += loss.item()
#             loss.backward()

#             torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

#             optimizer.step()
#             scheduler.step()

#         avg_train_loss = total_train_loss / len(train_dataloader)
#         training_time = format_time(time.time() - t0)

#         print("")
#         print("  Average training loss: {0:.2f}".format(avg_train_loss))
#         print("  Training epcoh took: {:}".format(training_time))

#         print("")
#         print("Running Validation...")

#         t0 = time.time()
#         model.eval()

#         total_eval_accuracy = 0
#         total_eval_loss = 0
#         nb_eval_steps = 0

#         for batch in validation_dataloader:
#             b_input_ids = batch[0].to(device)
#             b_input_mask = batch[1].to(device)
#             b_labels = batch[2].to(device)

#             with torch.no_grad():
#                 res = model(
#                     b_input_ids,
#                     token_type_ids=None,
#                     attention_mask=b_input_mask,
#                     labels=b_labels
#                 )
#             loss = res['loss']
#             logits = res['logits']
#             total_eval_loss += loss.item()

#             logits = logits.detach().cpu().numpy()
#             label_ids = b_labels.to('cpu').numpy()

#             total_eval_accuracy += flat_accuracy(logits, label_ids)

#         avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
#         print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

#         avg_val_loss = total_eval_loss / len(validation_dataloader)
#         validation_time = format_time(time.time() - t0)

#         print("  Validation Loss: {0:.2f}".format(avg_val_loss))
#         print("  Validation took: {:}".format(validation_time))

#         training_stats.append(
#             {
#                 'epoch': epoch_i + 1,
#                 'Training Loss': avg_train_loss,
#                 'Valid. Loss': avg_val_loss,
#                 'Valid. Accur.': avg_val_accuracy,
#                 'Training Time': training_time,
#                 'Validation Time': validation_time
#             }
#         )

#     print("")
#     print("Training complete!")
#     print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

#     print_header('display stats')
#     df_stats = pd.DataFrame(data=training_stats)
#     df_stats = df_stats.set_index('epoch')
#     print(df_stats)

#     print('Display in plot')
#     sns.set(style='darkgrid')
#     sns.set(font_scale=1.5)
#     plt.rcParams['figure.figsize'] = (12, 6)
#     plt.plot(df_stats['Training Loss'], 'b-o', label="Training")
#     plt.plot(df_stats['Valid. Loss'], 'g-o', label="Validation")

#     plt.title("Training & Validation Loss")
#     plt.xlabel("Epoch")
#     plt.ylabel("Loss")
#     plt.legend()
#     plt.xticks([1, 2, 3, 4])

#     plt.show()